In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.applications import resnet50
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout

# Transfer Learning

## Create initial resnet50 model

In [3]:
model = resnet50.ResNet50(include_top=True,weights='imagenet')

In [4]:
len(model.layers)

177

In [5]:
model.layers[:3], model.layers[-3:]

([<keras.engine.input_layer.InputLayer at 0x17360a49460>,
  <keras.layers.core.dense.Dense at 0x1736bbbb460>])

In [6]:
model.input.shape, model.output.shape

(TensorShape([None, 224, 224, 3]), TensorShape([None, 1000]))

## Duplicate the resnet50 using `Model`

In [7]:
model2 = Model(inputs=model.input, outputs=model.output)

In [8]:
len(model2.layers)

177

In [9]:
model2.input.shape, model2.output.shape

(TensorShape([None, 224, 224, 3]), TensorShape([None, 1000]))

## Extract resnet50 convolutional head

In [10]:
last_but_one_layer_output = model.layers[-2].output
last_but_one_layer_output

<KerasTensor: shape=(None, 2048) dtype=float32 (created by layer 'avg_pool')>

In [11]:
conv_head = Model(inputs=model.input, outputs=last_but_one_layer_output)

In [12]:
len(conv_head.layers)

176

In [13]:
conv_head.input.shape, conv_head.output.shape

(TensorShape([None, 224, 224, 3]), TensorShape([None, 2048]))

The output shape has changed to the 2048-element vector that avg pooling outputs.

## Heading

Natural images dataset https://www.kaggle.com/datasets/prasunroy/natural-images

The purpose is to use the brains of ResNet50, but classify the 8 classes in the Natural images dataset (instead of the 1000 classes in ResNet50's last layer).

Two ways to **create models**, two Keras APIs to **connect models and layers**:
- Keras Sequential API
- Keras Functional API

Keras Sequential API does not always work.


See more here https://www.tensorflow.org/guide/keras/functional

### Keras Sequential API

In [14]:
# Transfer learning model 1
tl_model1 = Sequential([
    conv_head,
    Dense(8, activation='softmax'),
])

tl_model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_1 (Functional)        (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 8)                 16392     
                                                                 
Total params: 23,604,104
Trainable params: 23,550,984
Non-trainable params: 53,120
_________________________________________________________________


### Keras Functional API

In [15]:
dense_layer = Dense(8, activation='softmax')

# TypeError: Inputs to a layer should be tensors.
# dense_layer(conv_head)

# Correct
dense_layer_output = dense_layer(conv_head.output)
dense_layer_output

<KerasTensor: shape=(None, 8) dtype=float32 (created by layer 'dense_1')>

### More functional API

Add few more layers. Навързване.

In [16]:
dense_intermediate = Dense(512, activation='relu')(conv_head.output)
dropout_final = Dropout(0.2)(dense_intermediate)
dense_final = Dense(9, activation='softmax')(dropout_final)

dense_final

<KerasTensor: shape=(None, 9) dtype=float32 (created by layer 'dense_3')>

## Freezing layers

In [17]:
transfer_learning = Sequential([
    conv_head,
    Dense(512, activation='relu'),
    Dropout(0.2),
    Dense(8, activation='softmax'),
])

transfer_learning.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_1 (Functional)        (None, 2048)              23587712  
                                                                 
 dense_4 (Dense)             (None, 512)               1049088   
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 8)                 4104      
                                                                 
Total params: 24,640,904
Trainable params: 24,587,784
Non-trainable params: 53,120
_________________________________________________________________


### All of ResNet50

In [18]:
for layer in conv_head.layers:
    layer.trainable = False
    
transfer_learning = Sequential([
    conv_head,
    Dense(512, activation='relu'),
    Dropout(0.2),
    Dense(8, activation='softmax'),
])

transfer_learning.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_1 (Functional)        (None, 2048)              23587712  
                                                                 
 dense_6 (Dense)             (None, 512)               1049088   
                                                                 
 dropout_2 (Dropout)         (None, 512)               0         
                                                                 
 dense_7 (Dense)             (None, 8)                 4104      
                                                                 
Total params: 24,640,904
Trainable params: 1,053,192
Non-trainable params: 23,587,712
_________________________________________________________________


### Part of ResNet50

In [21]:
for layer in conv_head.layers[:150]:
    layer.trainable = False

transfer_learning.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_3 (Functional)        (None, 2048)              23587712  
                                                                 
 dense_8 (Dense)             (None, 512)               1049088   
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_9 (Dense)             (None, 8)                 4104      
                                                                 
Total params: 24,640,904
Trainable params: 1,053,192
Non-trainable params: 23,587,712
_________________________________________________________________
